In [5]:
import tensorflow as tf
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt

def knnVote():
    '''
    KNN using majority vote
    '''
    #nearest indices
    A = tf.constant([1, 1, 2, 4, 4, 4, 7, 8, 8])
    print(A.shape)

    nearest_k_y, idx, votes = tf.unique_with_counts(A)
    print("y", nearest_k_y.eval())
    print("idx", idx.eval())
    print("votes", votes.eval())
    majority = tf.argmax(votes)
    predict_res = tf.gather(nearest_k_y, majority)
    
    
    print("majority", majority.eval())
    print("predict", predict_res.eval())
    return predict_res

init = tf.global_variables_initializer()
sessMain = tf.InteractiveSession()
sessMain.run(init)
knnVote()



/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


(9,)
y [1 2 4 7 8]
idx [0 0 1 2 2 2 3 4 4]
votes [2 1 3 1 2]
majority 2
predict 4


<tf.Tensor 'Gather:0' shape=() dtype=int32>

In [8]:
#def multidimensionalKnnVote():
A = tf.constant([
    [1, 1, 2, 4, 4, 4, 7, 8, 8],
    [2, 2, 3, 3, 3, 4, 4, 5, 6],
  ])

def cond(i, all_idxs, all_vals):
    return i < A.shape[0]

def body(i, all_idxs, all_vals):
    nearest_k_y, idx, votes = tf.unique_with_counts(A[i])
    majority_idx = tf.argmax(votes)
    majority_val = nearest_k_y[majority_idx]

    majority_idx = tf.reshape(majority_idx, shape=(1,))
    majority_val = tf.reshape(majority_val, shape=(1,))

    new_idxs = tf.cond(tf.equal(i, 0),
                       lambda: majority_idx,
                       lambda: tf.concat([all_idxs, majority_idx], axis=0))

    new_vals = tf.cond(tf.equal(i, 0),
                       lambda: majority_val,
                       lambda: tf.concat([all_vals, majority_val], axis=0))

    return i + 1, new_idxs, new_vals

# This means: starting from 0, apply the `body`, while the `cond` is true.
# Note that `shape_invariants` allow the 2nd and 3rd tensors to grow.
i0 = tf.constant(0)
idx0 = tf.constant(0, shape=(1,), dtype=tf.int64)
val0 = tf.constant(0, shape=(1,), dtype=tf.int32)
_, idxs, vals = tf.while_loop(cond, body,
                                loop_vars=(i0, idx0, val0),
                                shape_invariants=(i0.shape, tf.TensorShape([None]), tf.TensorShape([None])))

print('majority:', idxs.eval())
print('predict:', vals.eval())

majority: [2 1]
predict: [4 3]
